In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

def stuff_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

stuff_chain = retriever | RunnableLambda(stuff_docs)

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history",
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. "
            "Do not use any knowledge outside the provided context. "
            "If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": stuff_chain,
        "question": RunnablePassthrough(),
    }
    | RunnablePassthrough.assign(history=load_memory)
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)

questions = [
    "Is Aaronson guilty?",
    "What message did he write on the table?",
    "Who is Julia?",
]

for q in questions:
    invoke_chain(q)


Yes, according to the information provided, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.
He traced "2+2=5" with his finger in the dust on the table.
Julia is a character who was involved with Winston in a forbidden relationship.
